In [59]:
import csv
import pandas as pd
import numpy as np
import json
#with open("relacion_circunscripcion_colegio_sde.csv", "r", encoding="utf-8") as f:
#    data = csv.DictReader(f)

data = pd.read_csv("relacion_circunscripcion_colegio_sde.csv")


In [60]:

def reformat_colums(colegio):
     jsondata = pd.read_json(f"data/sde/colegio_{colegio}.json", dtype={"letra": str})
     print(jsondata["votosPreferenciales"][0])
     num_colums = len(jsondata["votosPreferenciales"][0])
     votos = pd.DataFrame(list(jsondata["votosPreferenciales"]))
     votos.columns = ["Casilla_{}".format(x) for x in range(1,num_colums +1)]
     new_votos = votos.map(lambda x: x["votos"])
     new_cat = pd.concat([jsondata,new_votos], axis=1)
     #print(new_cat.head())
     #print(jsondata.head())
     #print(new_cat["Casilla_4"].loc[2],"----", jsondata["votosPreferenciales"].loc[2])
     return new_cat




In [62]:
pda = reformat_colums(colegio="0001")
pdb = reformat_colums(colegio="0911A")
data_gen = (reformat_colums(x) for x in ["0001","0911A","2349"])
new_data = pd.concat(data_gen, ignore_index=True, )
new_data = new_data.fillna(-1)
new_data = new_data.astype({"Casilla_10": np.int64, })

[{'titulo': '1', 'votos': 8}, {'titulo': '2', 'votos': 21}, {'titulo': '3', 'votos': 7}, {'titulo': '4', 'votos': 1}, {'titulo': '5', 'votos': 3}, {'titulo': '6', 'votos': 4}, {'titulo': '7', 'votos': 3}, {'titulo': '8', 'votos': 1}, {'titulo': '9', 'votos': 4}, {'titulo': '10', 'votos': 0}, {'titulo': '11', 'votos': 1}, {'titulo': '12', 'votos': 0}]
[{'titulo': '1', 'votos': 4}, {'titulo': '2', 'votos': 16}, {'titulo': '3', 'votos': 5}, {'titulo': '4', 'votos': 2}, {'titulo': '5', 'votos': 21}, {'titulo': '6', 'votos': 1}, {'titulo': '7', 'votos': 4}, {'titulo': '8', 'votos': 2}, {'titulo': '9', 'votos': 2}]
[{'titulo': '1', 'votos': 8}, {'titulo': '2', 'votos': 21}, {'titulo': '3', 'votos': 7}, {'titulo': '4', 'votos': 1}, {'titulo': '5', 'votos': 3}, {'titulo': '6', 'votos': 4}, {'titulo': '7', 'votos': 3}, {'titulo': '8', 'votos': 1}, {'titulo': '9', 'votos': 4}, {'titulo': '10', 'votos': 0}, {'titulo': '11', 'votos': 1}, {'titulo': '12', 'votos': 0}]
[{'titulo': '1', 'votos': 4}, 

In [ ]:
pd.set_option('display.max_colwidth', None)
subset = new_data.loc[:, "Casilla_1"]
new_data[["letra", "siglas", "votosPreferenciales", "Casilla_1", "Casilla_6"]].loc[subset > 0]


,letra,siglas,votosPreferenciales,Casilla_1,Casilla_6
0,0001,PRM,"[{'titulo': '1', 'votos': 8}, {'titulo': '2', 'votos': 21}, {'titulo': '3', 'votos': 7}, {'titulo': '4', 'votos': 1}, {'titulo': '5', 'votos': 3}, {'titulo': '6', 'votos': 4}, {'titulo': '7', 'votos': 3}, {'titulo': '8', 'votos': 1}, {'titulo': '9', 'votos': 4}, {'titulo': '10', 'votos': 0}, {'titulo': '11', 'votos': 1}, {'titulo': '12', 'votos': 0}]",8,4
1,0001,PLD,"[{'titulo': '1', 'votos': 5}, {'titulo': '2', 'votos': 9}, {'titulo': '3', 'votos': 4}, {'titulo': '4', 'votos': 1}, {'titulo': '5', 'votos': 1}, {'titulo': '6', 'votos': 0}, {'titulo': '7', 'votos': 1}, {'titulo': '8', 'votos': 0}, {'titulo': '9', 'votos': 0}, {'titulo': '10', 'votos': 0}, {'titulo': '11', 'votos': 0}, {'titulo': '12', 'votos': 1}]",5,0
2,0001,FP,"[{'titulo': '1', 'votos': 2}, {'titulo': '2', 'votos': 0}, {'titulo': '3', 'votos': 1}, {'titulo': '4', 'votos': 17}, {'titulo': '5', 'votos': 0}, {'titulo': '6', 'votos': 0}, {'titulo': '7', 'votos': 1}, {'titulo': '8', 'votos': 2}, {'titulo': '9', 'votos': 1}, {'titulo': '10', 'votos': 0}, {'titulo': '11', 'votos': 0}, {'titulo': '12', 'votos': 0}]",2,0
31,0001,PED,"[{'titulo': '1', 'votos': 1}, {'titulo': '2', 'votos': 0}, {'titulo': '3', 'votos': 0}, {'titulo': '4', 'votos': 0}, {'titulo': '5', 'votos': 0}, {'titulo': '6', 'votos': 0}, {'titulo': '7', 'votos': 0}, {'titulo': '8', 'votos': 0}, {'titulo': '9', 'votos': 0}, {'titulo': '10', 'votos': 0}, {'titulo': '11', 'votos': 0}, {'titulo': '12', 'votos': 0}]",1,0
34,0911A,PRM,"[{'titulo': '1', 'votos': 4}, {'titulo': '2', 'votos': 16}, {'titulo': '3', 'votos': 5}, {'titulo': '4', 'votos': 2}, {'titulo': '5', 'votos': 21}, {'titulo': '6', 'votos': 1}, {'titulo': '7', 'votos': 4}, {'titulo': '8', 'votos': 2}, {'titulo': '9', 'votos': 2}]",4,1
35,0911A,PLD,"[{'titulo': '1', 'votos': 4}, {'titulo': '2', 'votos': 1}, {'titulo': '3', 'votos': 11}, {'titulo': '4', 'votos': 0}, {'titulo': '5', 'votos': 6}, {'titulo': '6', 'votos': 1}, {'titulo': '7', 'votos': 13}, {'titulo': '8', 'votos': 0}, {'titulo': '9', 'votos': 2}]",4,1
44,0911A,PCR,"[{'titulo': '1', 'votos': 1}, {'titulo': '2', 'votos': 0}, {'titulo': '3', 'votos': 0}, {'titulo': '4', 'votos': 0}, {'titulo': '5', 'votos': 0}, {'titulo': '6', 'votos': 0}, {'titulo': '7', 'votos': 0}, {'titulo': '8', 'votos': 0}, {'titulo': '9', 'votos': 0}]",1,0
62,0911A,OD,"[{'titulo': '1', 'votos': 4}, {'titulo': '2', 'votos': 0}, {'titulo': '3', 'votos': 0}, {'titulo': '4', 'votos': 0}, {'titulo': '5', 'votos': 0}, {'titulo': '6', 'votos': 0}, {'titulo': '7', 'votos': 0}, {'titulo': '8', 'votos': 0}, {'titulo': '9', 'votos': 0}]",4,0
68,2349,PRM,"[{'titulo': '1', 'votos': 16}, {'titulo': '2', 'votos': 6}, {'titulo': '3', 'votos': 0}, {'titulo': '4', 'votos': 4}, {'titulo': '5', 'votos': 1}, {'titulo': '6', 'votos': 1}, {'titulo': '7', 'votos': 0}, {'titulo': '8', 'votos': 0}, {'titulo': '9', 'votos': 4}, {'titulo': '10', 'votos': 1}, {'titulo': '11', 'votos': 1}, {'titulo': '12', 'votos': 1}]",16,1
69,2349,PLD,"[{'titulo': '1', 'votos': 23}, {'titulo': '2', 'votos': 4}, {'titulo': '3', 'votos': 4}, {'titulo': '4', 'votos': 1}, {'titulo': '5', 'votos': 0}, {'titulo': '6', 'votos': 1}, {'titulo': '7', 'votos': 2}, {'titulo': '8', 'votos': 0}, {'titulo': '9', 'votos': 0}, {'titulo': '10', 'votos': 0}, {'titulo': '11', 'votos': 0}, {'titulo': '12', 'votos': 0}]",23,1
